# Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Base Model

base_model_id = 'FacebookAI/roberta-base'


seed = 2024

# Training
num_train_epochs=10
batch_size = 32
learning_rate = 5e-5

# Regularisation
hidden_dropout_prob=0.2
attention_probs_dropout_prob=0.0
weight_decay=0.001

# Evaluation
label_threshold=0.5

use_gradient_checkpointing = True,  # Save some memory at the expense of training
# See https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one

hf_site_id = '2024-mcm-everitt-ryan'
dataset_id = f'{hf_site_id}/job-bias-synthetic-human-benchmark'

#dataset_id = f'{hf_site_id}/job-bias-synthetic-human-verified'


base_model_name = base_model_id.split('/')[-1]
model_id = f'{base_model_name}-job-bias-mixed'
hub_model_id = f'{hf_site_id}/{model_id}'

In [2]:
!pip install -q transformers datasets sentencepiece accelerate evaluate hf_transfer huggingface_hub scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip


# Dataset

In [3]:
from datasets import load_dataset

dataset = load_dataset(dataset_id)
column_names = dataset['train'].column_names


text_col = 'text'
label_cols = [col for col in column_names if col.startswith('label_')]

labels = [label.replace("label_", "") for label in label_cols]

id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

# Remove all columns apart from the two needed for multi-class classification
keep_columns = ['id', text_col] + label_cols
for split in ["train", "val", "test"]:
    dataset[split] = dataset[split].remove_columns(
        [col for col in dataset[split].column_names if col not in keep_columns])

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'label_age', 'label_disability', 'label_masculine', 'label_feminine', 'label_racial', 'label_sexuality', 'label_general', 'text'],
        num_rows: 4820
    })
    val: Dataset({
        features: ['id', 'label_age', 'label_disability', 'label_masculine', 'label_feminine', 'label_racial', 'label_sexuality', 'label_general', 'text'],
        num_rows: 1051
    })
    test: Dataset({
        features: ['id', 'label_age', 'label_disability', 'label_masculine', 'label_feminine', 'label_racial', 'label_sexuality', 'label_general', 'text'],
        num_rows: 1053
    })
})

In [4]:
import pandas as pd

# Merge train,val, test into one dataframe
df = pd.concat([
    dataset['train'].to_pandas(),
    dataset['val'].to_pandas(),
    dataset['test'].to_pandas()])

df.head(3)

,id,label_age,label_disability,label_masculine,label_feminine,label_racial,label_sexuality,label_general,text
0,Synthetic:meta-llama:Meta-Llama-3-70B-Instruct...,False,False,False,False,True,False,False,"Company: Harrington, Richardson and Collins\n\..."
1,Synthetic:meta-llama:Meta-Llama-3-70B-Instruct...,False,False,False,False,True,False,False,Part Time Checker at Allen Inc\n\nCompany Back...
2,Synthetic:gpt-4o-2024-05-13:20240627151822:3e7...,False,False,False,False,True,False,False,Torres-Spencer is a leading firm in the transp...


In [5]:
# Longest phrase
longest_text = df[text_col].apply(lambda x: (len(x), x)).max()[1]
longest_text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Job Posting</title>\n</head>\n<body>\n\n<h1>Software Developer - CodeWave Solutions - Canada</h1>\n\n<p><strong>Company Background:</strong></p>\n<p>CodeWave Solutions is a leading software development firm based in Toronto, Canada. We specialize in delivering innovative software solutions to clients across various industries. Our commitment to excellence and passion for technology drives us to create cutting-edge applications that power businesses worldwide.</p>\n\n<p><strong>Job Type:</strong> Full-time</p>\n\n<p><strong>Job Description:</strong></p>\n<p>As a Software Developer at CodeWave Solutions, you will play a key role in designing, developing, and deploying high-quality software solutions. We are seeking knowledgeable candidates who are enthusiastic about technology and have a strong commitment to producing top-notch wor

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_prefix_space=True)
tokenizer

RobertaTokenizerFast(name_or_path='FacebookAI/roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [7]:
max_char = len(longest_text)
max_words = len(longest_text.split())
max_tokens = len(tokenizer.encode(longest_text))

print(f'Max characters: {max_char}')
print(f'Max words: {max_words}')
print(f'Max tokens: {max_tokens}')

Token indices sequence length is longer than the specified maximum sequence length for this model (1034 > 512). Running this sequence through the model will result in indexing errors


Max characters: 3863
Max words: 441
Max tokens: 1034


In [8]:
tokenizer_max_length = min(max_tokens, tokenizer.model_max_length)
tokenizer_max_length

512

In [9]:
import numpy as np


def preprocess_data(sample):
    # take a batch of texts
    text = sample[text_col]
    # encode them
    encoding = tokenizer(text, truncation=True, max_length=tokenizer_max_length, padding="max_length")
    #encoding = tokenizer(text, truncation=True, max_length=tokenizer_max_length, padding=True)
    # add labels
    labels_batch = {k: sample[k] for k in sample.keys() if k in label_cols}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(label_cols)))
    # fill numpy array
    for idx, label in enumerate(label_cols):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [10]:
#ds_train = ds_train.map(tokenize, batched=True, batch_size=len(ds_train))
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
encoded_dataset.set_format("torch")

Map:   0%|          | 0/4820 [00:00<?, ? examples/s]

Map:   0%|          | 0/1051 [00:00<?, ? examples/s]

Map:   0%|          | 0/1053 [00:00<?, ? examples/s]

# Model

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(base_model_id,
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(label_cols),
                                                           id2label=id2label,
                                                           label2id=label2id,
                                                           hidden_dropout_prob=hidden_dropout_prob,
                                                          # attention_probs_dropout_prob=attention_probs_dropout_prob
                                                          )
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [12]:
model.config

RobertaConfig {
  "_name_or_path": "FacebookAI/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "id2label": {
    "0": "age",
    "1": "disability",
    "2": "masculine",
    "3": "feminine",
    "4": "racial",
    "5": "sexuality",
    "6": "general"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "age": 0,
    "disability": 1,
    "feminine": 3,
    "general": 6,
    "masculine": 2,
    "racial": 4,
    "sexuality": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "transformers_version": "4.42.3",
  "type_vocab_size": 1,
  "use_cache": t

# Define Metrics

In [13]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score, classification_report
from transformers import EvalPrediction
import torch


# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
# added extras
def multi_label_metrics(predictions, labels):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= label_threshold)] = 1
    # finally, compute metrics
    y_true = labels

    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    
    f1_micro = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_samples = f1_score(y_true=y_true, y_pred=y_pred, average='samples')
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')

    precision_micro = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
    recall_micro = recall_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc_micro = roc_auc_score(y_true=y_true, y_score=y_pred, average='micro')

    print(classification_report(y_true, y_pred, target_names=list(id2label.values())))
    
    # return as dictionary
    metrics = {
        'accuracy': accuracy,
        f'f1_micro': f1_micro,
        f'f1_macro': f1_macro,
        f'f1_samples': f1_samples,
        f'f1_weighted': f1_weighted,
        f'precision_micro': precision_micro,
        f'recall_micro': recall_micro,
        f'roc_auc_micro': roc_auc_micro}
    return metrics


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

# Train

In [14]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding,TrainerCallback
from huggingface_hub import HfFolder

metric_name = 'loss' #"f1_micro"

args = TrainingArguments(
    model_id,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    #optim=optimiser,
    #lr_scheduler_type="cosine",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    fp16=True,
    gradient_checkpointing=use_gradient_checkpointing,
    overwrite_output_dir=True,
    #push_to_hub=True,
    #output_dir=repository_id,
    #logging_dir=f"{model_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    #warmup_steps=500,
    #warmup_ratio=0.1,
    #max_grad_norm=0.3,
    save_total_limit=3,
    #report_to="tensorboard",
    #push_to_hub=True,
    #hub_strategy="every_save",
    #hub_model_id=hub_model_id,
    #hub_token=HfFolder.get_token(),
)

#early_stop = transformers.EarlyStoppingCallback(10, 1.15)
class PrintClassificationCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, logs=None, **kwargs):
        print("----------------------------------------------------------")


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["val"],
    # For padding a batch of examples to the maximum length seen in the batch
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[PrintClassificationCallback]
    #tokenizer=tokenizer,
    #   callbacks=[early_stop]
)

model.config.use_cache = False  # Silence the warnings.
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Samples,F1 Weighted,Precision Micro,Recall Micro,Roc Auc Micro
1,0.445000,0.371200,0.394862,0.062212,0.019882,0.020742,0.020095,0.087948,0.048128,0.503464
2,0.244700,0.311724,0.463368,0.444013,0.458406,0.280196,0.457719,0.348577,0.611408,0.758544
3,0.173900,0.206456,0.622265,0.604167,0.623859,0.339219,0.623323,0.518519,0.723708,0.834117
4,0.131900,0.201176,0.618459,0.617284,0.651745,0.357283,0.651395,0.520833,0.757576,0.850021
5,0.125900,0.186616,0.688868,0.645425,0.674091,0.332921,0.673580,0.595777,0.704100,0.832332
6,0.120700,0.224549,0.638440,0.628364,0.669964,0.356237,0.669465,0.530713,0.770053,0.856922
7,0.077700,0.214833,0.647003,0.633793,0.661391,0.356264,0.660803,0.534230,0.778966,0.861452
8,0.094000,0.218495,0.647954,0.639659,0.666534,0.370599,0.665953,0.531915,0.802139,0.871935
9,0.073300,0.214636,0.655566,0.640227,0.662568,0.369100,0.662145,0.531140,0.805704,0.873497
10,0.056300,0.201147,0.674596,0.651786,0.673644,0.360863,0.673186,0.559387,0.780749,0.864992


              precision    recall  f1-score   support

         age       0.00      0.00      0.00        80
  disability       0.00      0.00      0.00        81
   masculine       0.00      0.00      0.00        81
    feminine       0.00      0.00      0.00        80
      racial       0.00      0.00      0.00        77
   sexuality       0.00      0.00      0.00        81
     general       0.09      0.33      0.14        81

   micro avg       0.09      0.05      0.06       561
   macro avg       0.01      0.05      0.02       561
weighted avg       0.01      0.05      0.02       561
 samples avg       0.03      0.02      0.02       561

----------------------------------------------------------
              precision    recall  f1-score   support

         age       0.51      0.45      0.48        80
  disability       0.23      0.80      0.36        81
   masculine       0.54      0.36      0.43        81
    feminine       0.55      0.90      0.69        80
      racial       

TrainOutput(global_step=1510, training_loss=0.16947846308054512, metrics={'train_runtime': 560.5082, 'train_samples_per_second': 85.993, 'train_steps_per_second': 2.694, 'total_flos': 1.268252219904e+16, 'train_loss': 0.16947846308054512, 'epoch': 10.0})

# Evaluate

In [15]:
test_results = trainer.evaluate(eval_dataset=encoded_dataset['test'])
test_results

              precision    recall  f1-score   support

         age       0.56      0.69      0.62        81
  disability       0.64      0.60      0.62        81
   masculine       0.80      0.63      0.70        81
    feminine       0.89      0.88      0.88        81
      racial       0.58      0.88      0.70        78
   sexuality       0.78      0.81      0.80        81
     general       0.25      0.46      0.32        82

   micro avg       0.59      0.71      0.64       565
   macro avg       0.64      0.71      0.66       565
weighted avg       0.64      0.71      0.66       565
 samples avg       0.33      0.36      0.34       565

----------------------------------------------------------


{'eval_loss': 0.2010984867811203,
 'eval_accuracy': 0.6970560303893637,
 'eval_f1_micro': 0.6436041834271923,
 'eval_f1_macro': 0.6633665915527766,
 'eval_f1_samples': 0.33818116040338264,
 'eval_f1_weighted': 0.6625676793329273,
 'eval_precision_micro': 0.5899705014749262,
 'eval_recall_micro': 0.7079646017699115,
 'eval_roc_auc_micro': 0.8335591448461664,
 'eval_runtime': 2.7148,
 'eval_samples_per_second': 387.872,
 'eval_steps_per_second': 12.156,
 'epoch': 10.0}

In [16]:
import pandas as pd
df = pd.DataFrame(list(test_results.items()), columns=['Metric', 'Value'])
print(df.to_string(index=False))

                 Metric      Value
              eval_loss   0.201098
          eval_accuracy   0.697056
          eval_f1_micro   0.643604
          eval_f1_macro   0.663367
        eval_f1_samples   0.338181
       eval_f1_weighted   0.662568
   eval_precision_micro   0.589971
      eval_recall_micro   0.707965
     eval_roc_auc_micro   0.833559
           eval_runtime   2.714800
eval_samples_per_second 387.872000
  eval_steps_per_second  12.156000
                  epoch  10.000000


# Push to Hugging Face

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:

from huggingface_hub import ModelCard, EvalResult, ModelCardData
import platform
import sys
import os

model.push_to_hub(repo_id=hub_model_id, token=HfFolder.get_token())
tokenizer.push_to_hub(repo_id=hub_model_id, token=HfFolder.get_token())

###### Update Model Card ######

eval_results = []
for k, v in test_results.items():
    eval_results.append(EvalResult(
        task_type='multi_label_classification',
        dataset_type='mix_human-eval_synthetic',
        dataset_name=dataset_id,
        metric_type=k.replace("eval_", "", 1),
        metric_value=v))

direct_use = """
    ```python
    from transformers import pipeline

    pipe = pipeline("text-classification", model="${hub_model_id}", return_all_scores=True)

    results = pipe("Join our dynamic and fast-paced team as a Junior Marketing Specialist. We seek a tech-savvy and energetic individual who thrives in a vibrant environment. Ideal candidates are digital natives with a fresh perspective, ready to adapt quickly to new trends. You should have recent experience in social media strategies and a strong understanding of current digital marketing tools. We're looking for someone with a youthful mindset, eager to bring innovative ideas to our young and ambitious team. If you're a recent graduate or early in your career, this opportunity is perfect for you!")
    print(results)
    ```
    >> [[
    {'label': 'age', 'score': 0.9883460402488708}, 
    {'label': 'disability', 'score': 0.00787709467113018}, 
    {'label': 'feminine', 'score': 0.007224376779049635}, 
    {'label': 'general', 'score': 0.09967829287052155}, 
    {'label': 'masculine', 'score': 0.0035264550242573023}, 
    {'label': 'racial', 'score': 0.014618005603551865}, 
    {'label': 'sexuality', 'score': 0.005568435415625572}
    ]]
    """
direct_use = direct_use.replace('${hub_model_id}', hub_model_id, -1)

card_data = ModelCardData(
    model_id=model_id,
    model_name=model_id,
    model_description="The model is a multi-label classifier designed to detect various types of bias within job descriptions.",
    base_model=base_model_id,
    language='en',
    license='apache-2.0',
    developers="Tristan Everitt and Paul Ryan",
    model_card_authors='See developers',
    model_card_contact='See developers',
    repo="https://gitlab.computing.dcu.ie/everitt2/2024-mcm-everitt-ryan",
    eval_results=eval_results,
    compute_infrastructure=f'{platform.system()} {platform.release()} {platform.processor()}',
    # hardware_requirements=f"CPUs: {psutil.cpu_count()}, Memory: {psutil.virtual_memory().total} bytes",
    software=f'Python {platform.python_version()}',
    hardware_type=platform.machine(),
    hours_used='N/A',
    cloud_provider='N/A',
    cloud_region='N/A',
    co2_emitted='N/A',
    datasets=[dataset_id],
    direct_use=direct_use
)

card = ModelCard.from_template(card_data)

card.push_to_hub(repo_id=hub_model_id, token=HfFolder.get_token())

README.md:   0%|          | 0.00/9.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/2024-mcm-everitt-ryan/roberta-base-job-bias-mixed/commit/39627f99f699f1fd427b4688d758b77fa5ed171a', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='39627f99f699f1fd427b4688d758b77fa5ed171a', pr_url=None, pr_revision=None, pr_num=None)